In [1]:
'''В ноутбуке используются модули HTML и библиотека Plotly,поэтому рекомендуется запускать ноутбук именно в браузере,а не в jupyter lab,
потому как данные библиотеки наилучшим образом показывают себя именно в браузере без всяких велосипедов.У меня в ноутбуке используется ширина 
ноутбука 70 или 80% от ширины страницы,поэтому не знаю как оно будет отображаться в ноутбуке со стандартной шириной в настройках '''

'В ноутбуке используются модули HTML и библиотека Plotly,поэтому рекомендуется запускать ноутбук именно в браузере,а не в jupyter lab,\nпотому как данные библиотеки наилучшим образом показывают себя именно в браузере без всяких велосипедов.У меня в ноутбуке используется ширина \nноутбука 70 или 80% от ширины страницы,поэтому не знаю как оно будет отображаться в ноутбуке со стандартной шириной в настройках '

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from IPython.core.display import display, HTML
from plotly.offline import iplot, download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)
import itertools as itr
from statsmodels.stats import weightstats 

In [3]:
# загрузим датафрейм
df = pd.read_csv('stud_math.csv')
# посмотрим на название колонок,чтобы не было проблем при обращении
df.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'studytime, granular', 'higher', 'internet', 'romantic', 'famrel',
       'freetime', 'goout', 'health', 'absences', 'score'],
      dtype='object')

In [4]:
# имеется непонятная колонка 'studytime, granular',которой не было в описании датафрейма .Дропнем ее
df = df.drop(['studytime, granular'], axis=1)
df.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout',
       'health', 'absences', 'score'],
      dtype='object')

In [5]:
df.info()  # Обзор

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 29 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   school      395 non-null    object 
 1   sex         395 non-null    object 
 2   age         395 non-null    int64  
 3   address     378 non-null    object 
 4   famsize     368 non-null    object 
 5   Pstatus     350 non-null    object 
 6   Medu        392 non-null    float64
 7   Fedu        371 non-null    float64
 8   Mjob        376 non-null    object 
 9   Fjob        359 non-null    object 
 10  reason      378 non-null    object 
 11  guardian    364 non-null    object 
 12  traveltime  367 non-null    float64
 13  studytime   388 non-null    float64
 14  failures    373 non-null    float64
 15  schoolsup   386 non-null    object 
 16  famsup      356 non-null    object 
 17  paid        355 non-null    object 
 18  activities  381 non-null    object 
 19  nursery     379 non-null    o

In [6]:
'''визуализируем по каждому столбцу гистограмму содержащихся типов'''
number = len(df.columns)
fig = make_subplots(rows=1, cols=number, column_titles=list(df.columns))
fig.update_layout(width=2500, height=800, showlegend=False)
objects = [go.Histogram(x=df[columns].apply(lambda x: str(type(x))), name=columns)
           for columns in df.columns]
fig.add_traces(objects,
               rows=[1]*len(df.columns),
               cols=[i for i in range(1, number+1, 1)])
# видим что во многих колонках объектных типов есть значения float.Я думаю это NаN

In [7]:
# Заменим NAN  и пропуски в объектных типах на None
for col in df.columns:
    if df[col].dtype == type(object()):
        df[col] = df[col].apply(lambda x: x if isinstance(x, str) else '')
        df[col] = df[col].apply(lambda x: None if x.strip() == '' else x)

In [ ]:
# глянем целиком что получилсоь
# по факту реальных числовых значений только 3-возраст,пропуски и оценка. Остальные категориальные,хоть многие и используют числа для маркировки
display(HTML(df.to_html()))

In [ ]:
'''построим распеределние нечисловых значений'''
objects = [go.Histogram(x=df[columns],
                        name=columns) for columns in
           df.columns if df[columns].dtype == type(object)]
fig = make_subplots(rows=1, cols=len(objects), subplot_titles=[columns for columns in df.columns if df[columns].
                                                               dtype == type(object())])
fig.update_layout(height=800, width=1600, showlegend=False)
fig.add_traces(objects,
               rows=[1]*len(objects),
               cols=[i for i in range(1, len(objects)+1, 1)])

In [ ]:
'''строим распеределение признаков для столбцов с категориальными данными,обозначенными числом значениями(сюда же
отнесем и возраст,хотя он и не совсем категориальный признак,но там мало значений)'''
names = [columns for columns in df.columns if (
    (df[columns].dtype != type(object())) & (columns not in ['absences', 'score']))]
objects = [go.Box(y=df[columns], name=columns) for columns in df.columns if (
    (df[columns].dtype != type(object())) & (columns not in ['absences', 'score']))]
fig = make_subplots(rows=1, cols=len(objects), column_titles=names)
fig.update_layout(height=800, width=1600, showlegend=False)
fig.add_traces(objects,
               rows=[1]*len(objects),
               cols=[i for i in range(1, len(objects)+1, 1)])

In [ ]:
'''теперь строим распеределение признаков для столбцов с истинными числовыми данными'''
names = [columns for columns in df.columns if (
    (df[columns].dtype != type(object())) & (columns in ['absences', 'score']))]
objects = [go.Box(y=df[columns], name=columns) for columns in df.columns if (
    (df[columns].dtype != type(object())) & (columns in ['absences', 'score']))]
fig = make_subplots(rows=1, cols=len(objects), column_titles=names)
fig.update_layout(height=800, width=1600, showlegend=False)
fig.add_traces(objects,
               rows=[1]*len(objects),
               cols=[i for i in range(1, len(objects)+1, 1)])

In [ ]:
"теперь оценим графики и посмотрим что необходимо сделать"
# 2.Absences-действительно много серьезных выбросов,что скорее указывает на ошибку,т.к к примеру врядли человек с пропусками 385 дней набрал бы 85 баллов
# 17 выбросов,думаю стоит их убрать
display(df['absences'][df['absences'] > 20].value_counts().sum())
# 3.Score-Тут тоже не ясно.Мне кажется 0 балов невозможно набрать.сколько таких?
display(df['score'][df['score'] == 0].value_counts())
# 37 человек.Не многовато ли?думаю стоит выкинуть
df['absences'] = df['absences'].apply(
    lambda x: np.nan if x > 20 else x)  # заменим их на NaN
df['score'] = df['score'].apply(lambda x: np.nan if x == 0 else x)
df = df.dropna(subset=['absences', 'score'], axis=0)  # и дропнем
# Fedu-есть значение 40,заменим его на 4
df['Fedu'] = df['Fedu'].apply(lambda x: x if x != 40 else 4)
# famrel есть значение -1,заменим на 1
df['famrel'] = df['famrel'].apply(lambda x: x if x != -1 else 1)
# age удалим выброс 22 и заменим на медиану
df['age'] = df['age'].apply(lambda x: x if x != 22 else 17)

In [ ]:
# корреляции между числовыми признаками не заметно
df[['absences', 'score']].corr()

In [ ]:
fig = px.scatter_matrix(df[['absences', 'score']])
fig.update_layout(width=800, height=800)

In [ ]:
'''визуализируем распределение score для категориальных данных'''
columns=[cols for cols in df.columns if cols not in ['absences', 'score']]
for i,column in enumerate(columns):
    val=list(df[column].value_counts().index)
    fig=make_subplots(rows=1,cols=1,subplot_titles=[column])
    fig.update_layout(height=350)
    objects=[go.Box(y=df['score'][df[column]==el],name=el) for el in val]
    fig.add_traces(objects)
    fig.show()

In [ ]:
# визуально вроде есть различия ,Но подойдем более предметно и проверим гипотезы об одинаковом распеределении Score в зависимости от значнеий категориальных данных

In [ ]:
'''Функция,определяющая гипотезу разности параметров H0:X1==X2; Ha=X1!=X2'''


def get_stat_dif(column):
    val = list(df[column].dropna().value_counts().index)
    combine = list(itr.combinations(val, 2))
    for combs in combine:
        x1 = df['score'][df[column] == combs[0]]
        x2 = df['score'][df[column] == combs[1]]
        if weightstats.ttest_ind(x1.dropna(), x2.dropna())[1] < 0.05:
            return column
            break


# применим ее в цикле нечисловых колонок
columns = [cols for cols in df.columns if cols not in ['absences', 'score']]
znach = []  # пустой список куда запишем результат функции с названиями нужных колонок
for col in columns:
    get_stat_dif(col)
    znach.append(get_stat_dif(col))
znach = set(pd.Series(znach).dropna())  # преобразуем его во множество
# и сформируем список колонок кторые можно будет удалить через разность множеств
cols_list = list(set(columns)-znach)
cols_list

In [ ]:
'''таким образом можем создать датафрейм с колонками,которые нужны будут для модели'''
result = df.drop(cols_list, axis=1)
result = result.dropna()  # и удалим строки с пустыми значениями
result

In [ ]:
'''таким образом мы сформировали датасет для модели
1.В данных много пустых значений, и пришлось потерять почти половину в итоговом датафрейме
2.Выбросы найдены во многих столбцах,особенно они видимы в числовых столбцах
3.Были оценены как числовые,так и категориальные данные и сделаны выводы  о том,какие
столбцы необходимо использовать для модели
4.Понимание сферы,к которой относятся данные, является важным условием для их анализа.К примеру если бы данные были медицинские
то, я нашел бы возможность заменить пустые значения ,а не удалять строки с ними целиком,а также адекватно
смог бы оценить выбросы на предмет возможности такого выброса в реальности'''